# Задачи 2 и 3 се извршени со нецело податочно множество, поточно 1 милион линии од текст фајлот се испроцесирани за задача 2, и 30 илјади за 3-та, поради РАМ лимити или предолги тренирања. Ваков проблем немав со првата задача па затоа таа е решена со цел датасет.

In [ ]:
!7z x amazon-meta.txt.gz


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 210807517 bytes (202 MiB)

Extracting archive: amazon-meta.txt.gz
--
Path = amazon-meta.txt.gz
Type = gzip
Headers Size = 26

  0% - amazon-meta.txt                        2% - amazon-meta.txt                        4% - amazon-meta.txt                        6% - amazon-meta.txt                        8% - amazon-meta.txt                       10% - amazon-meta.txt                       12% - amazon-meta.txt                      

In [ ]:
!pip install torch==2.2.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 94.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.2.1+cu121.html

Looking in links: https://data.pyg.org/whl/torch-2.2.1+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 943.4/943.4 kB 18.6 MB/s eta 0:00:00


In [ ]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 30.3 MB/s eta 0:00:00


In [ ]:
!cp '/content/drive/MyDrive/Colab Notebooks/amazon-meta.txt' /content/

In [ ]:
import torch
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from torch_geometric.nn import to_hetero
from torch_geometric.datasets import IMDB
from torch_geometric.loader import NeighborLoader
from torch.nn.functional import dropout
from torch_geometric.nn import GCNConv, Linear, SAGEConv
from torch.optim import Adam
import networkx as nx

In [ ]:
class GCN(torch.nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        self.conv1 = SAGEConv((-1, -1), 64)
        self.conv2 = SAGEConv((-1, -1), 128)

        self.linear1 = Linear(128, 64)
        self.linear2 = Linear(64, num_classes)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).tanh()
        x = dropout(x, p=0.3)

        x = self.conv2(x, edge_index).tanh()
        x = dropout(x, p=0.3)

        x = self.linear1(x)
        x = self.linear2(x)

        return x

# Претпроцесирање на дата и креирање граф објект

In [ ]:
import numpy as np
import torch
from torch_geometric.data import HeteroData
from torch_geometric.loader import NeighborLoader
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from tqdm import tqdm

def process_and_create_hetero_data(file_path, limit=None):
    products_by_id = {}
    group_by_id = {}
    similar_by_id = defaultdict(list)
    reviewers_by_p_id = defaultdict(list)
    customer_map = {}
    asin_map = {}
    next_product_id = 0
    next_customer_id = 0
    groups = []

    with open(file_path, 'r') as file:
        total_lines = sum(1 for _ in file)

    with open(file_path, 'r') as file:
        current_asin = None
        for line_num, line in enumerate(tqdm(file, total=total_lines, desc="Processing file", unit="line")):
            if limit and line_num >= limit:
                break

            line = line.strip()
            if line.startswith("ASIN:"):
                current_asin = line.split("ASIN: ")[1]
                if current_asin not in asin_map:
                    asin_map[current_asin] = next_product_id
                    products_by_id[next_product_id] = current_asin
                    next_product_id += 1
            elif line.startswith("group:"):
                groups.append(line.split("group: ")[1])
                group_by_id[asin_map[current_asin]] = groups[-1]
            elif line.startswith("similar:"):
                similar_asins = line.split("similar: ")[1].split()[1:]
                for similar_asin in similar_asins:
                    if similar_asin not in asin_map:
                        asin_map[similar_asin] = next_product_id
                        products_by_id[next_product_id] = similar_asin
                        next_product_id += 1
                    similar_by_id[asin_map[current_asin]].append(asin_map[similar_asin])
            elif "cutomer:" in line:
                customer_info = line.split("cutomer: ")[1].split()
                customer_id, rating = customer_info[0], float(customer_info[2])
                if customer_id not in customer_map:
                    customer_map[customer_id] = next_customer_id
                    next_customer_id += 1
                reviewers_by_p_id[asin_map[current_asin]].append((customer_map[customer_id], rating))

    data = HeteroData()

    data['products'].x = torch.rand(len(products_by_id), 10)
    data['customers'].x = torch.rand(len(customer_map), 10)

    similar_edges = []
    for source, targets in similar_by_id.items():
        for target in targets:
            similar_edges.append((source, target))
    if similar_edges:
        data['products', 'similar_to', 'products'].edge_index = torch.tensor(similar_edges, dtype=torch.long).t().contiguous()

    review_edges = []
    review_ratings = []
    for product, reviews in reviewers_by_p_id.items():
        for customer, rating in reviews:
            review_edges.append((product, customer))
            review_ratings.append(rating)
    if review_edges:
        data['products', 'reviewed_by', 'customers'].edge_index = torch.tensor(review_edges, dtype=torch.long).t().contiguous()
        data['products', 'reviewed_by', 'customers'].edge_attr = torch.tensor(review_ratings, dtype=torch.float32)

    encoder = LabelEncoder()
    product_labels = encoder.fit_transform([group_by_id.get(pid, "") for pid in range(len(products_by_id))])
    data['products'].y = torch.tensor(product_labels, dtype=torch.long)

    num_products = len(products_by_id)
    indices = np.arange(num_products)
    np.random.shuffle(indices)

    train_cutoff = int(0.65 * num_products)
    test_cutoff = int(0.95 * num_products)

    train_mask = torch.zeros(num_products, dtype=torch.bool)
    val_mask = torch.zeros(num_products, dtype=torch.bool)
    test_mask = torch.zeros(num_products, dtype=torch.bool)

    train_mask[indices[:train_cutoff]] = True
    test_mask[indices[train_cutoff:test_cutoff]] = True
    val_mask[indices[test_cutoff:]] = True

    data['products'].train_mask = train_mask
    data['products'].test_mask = test_mask
    data['products'].val_mask = val_mask

    train_loader = NeighborLoader(data, num_neighbors=[3, 3], shuffle=True,
                                  input_nodes=('products', train_mask), batch_size=128)
    val_loader = NeighborLoader(data, num_neighbors=[3, 3], shuffle=False,
                                input_nodes=('products', val_mask), batch_size=128)
    test_loader = NeighborLoader(data, num_neighbors=[3, 3], shuffle=False,
                                 input_nodes=('products', test_mask), batch_size=128)

    data['products'].x = data['products'].x.float()
    data['customers'].x = data['customers'].x.float()
    for edge_type in data.edge_types:
        if 'edge_attr' in data[edge_type]:
            data[edge_type].edge_attr = data[edge_type].edge_attr.float()
    return data, train_loader, val_loader, test_loader

file_path = 'amazon-meta.txt'
data, train_loader, val_loader, test_loader = process_and_create_hetero_data(file_path, limit=30000)


Processing file:   0%|          | 30000/15010574 [00:00<00:44, 337324.49line/s]


# По процесирање на цел датасет, испаѓа дека се 11 класи(вклучувајќи ја и празен стринг класата), а не само 4-те наведени во фајлот.
# Оваа листа ја нема во кодот бидејќи ја добив со принтање на сите класи кои кодот погоре ги екстрахира, па потоа го ископирав аутпутот и го внесов како листа. Ќелијата го кочеше цел колаб и морав да ја избришам

In [ ]:
set(lista)

{'',
 'Baby Product',
 'Book',
 'CE',
 'DVD',
 'Music',
 'Software',
 'Sports',
 'Toy',
 'Video',
 'Video Games'}

In [ ]:
data

HeteroData(
  products={
    x=[721342, 10],
    y=[721342],
    train_mask=[721342],
    test_mask=[721342],
    val_mask=[721342],
  },
  customers={ x=[1555170, 10] },
  (products, similar_to, products)={ edge_index=[2, 1788725] },
  (products, reviewed_by, customers)={
    edge_index=[2, 7593244],
    edge_attr=[7593244],
  }
)

# Задача 1

In [ ]:
def train_classification(model,
                         train_loader, val_loader,
                         optimizer, criterion,
                         epochs=5):
    for epoch in range(epochs):
        for i, batch in enumerate(train_loader):
            model.train()
            optimizer.zero_grad()

            x_dict = batch.x_dict
            edge_index_dict = batch.edge_index_dict
            train_mask = batch['products'].train_mask
            data_y = batch['products'].y

            out = model(x_dict, edge_index_dict)
            loss = criterion(out['products'][train_mask], data_y[train_mask])
            train_loss = loss.item()

            loss.backward()
            optimizer.step()

            print(f'Epoch: {epoch:03d}, Step: {i:03d}, Loss: {train_loss:.4f}')

In [ ]:
base_model = GCN(num_classes=11)

In [ ]:
model = to_hetero(base_model, data.metadata(), aggr='sum')

/usr/local/lib/python3.10/dist-packages/torch_geometric/nn/fx.py:132: UserWarning: Found function 'dropout' with keyword argument 'training'. During FX tracing, this will likely be baked in as a constant value. Consider replacing this function by a module to properly encapsulate its training flag.
  warnings.warn(f"Found function '{node.name}' with keyword "
/usr/local/lib/python3.10/dist-packages/torch_geometric/nn/fx.py:132: UserWarning: Found function 'dropout_1' with keyword argument 'training'. During FX tracing, this will likely be baked in as a constant value. Consider replacing this function by a module to properly encapsulate its training flag.
  warnings.warn(f"Found function '{node.name}' with keyword "


In [ ]:
model = model.float()

In [ ]:
from torch.nn.functional import one_hot


def train_classification(model,
                         train_loader, val_loader,
                         optimizer, criterion,
                         epochs=5):
    for epoch in range(epochs):
        for i, batch in enumerate(train_loader):
            model.train()
            optimizer.zero_grad()

            x_dict = batch.x_dict
            edge_index_dict = batch.edge_index_dict
            train_mask = batch['products'].train_mask
            data_y = batch['products'].y

            out = model(x_dict, edge_index_dict)
            loss = criterion(out['products'][train_mask], data_y[train_mask])
            train_loss = loss.item()

            loss.backward()
            optimizer.step()

            print(f'Epoch: {epoch:03d}, Step: {i:03d}, Loss: {train_loss:.4f}')

        for i, batch in enumerate(val_loader):
            model.eval()

            x_dict = batch.x_dict
            edge_index_dict = batch.edge_index_dict
            val_mask = batch['products'].val_mask
            data_y = batch['products'].y

            out = model(x_dict, edge_index_dict)
            loss = criterion(out['products'][val_mask], data_y[val_mask])
            val_loss = loss.item()

            print(f'Epoch: {epoch:03d}, Step: {i:03d}, Val Loss: {val_loss:.4f}')

In [ ]:
optimizer = Adam(model.parameters(), lr=0.001)
criterion = CrossEntropyLoss()
train_classification(model=model,
                     train_loader=train_loader, val_loader=val_loader,
                     optimizer=optimizer, criterion=criterion,
                     epochs=10)

Streaming output truncated to the last 5000 lines.
Epoch: 008, Step: 2892, Loss: 0.9240
Epoch: 008, Step: 2893, Loss: 0.9095
Epoch: 008, Step: 2894, Loss: 1.0160
Epoch: 008, Step: 2895, Loss: 0.9837
Epoch: 008, Step: 2896, Loss: 0.8539
Epoch: 008, Step: 2897, Loss: 0.9708
Epoch: 008, Step: 2898, Loss: 0.8055
Epoch: 008, Step: 2899, Loss: 1.0800
Epoch: 008, Step: 2900, Loss: 0.8989
Epoch: 008, Step: 2901, Loss: 0.9798
Epoch: 008, Step: 2902, Loss: 0.9023
Epoch: 008, Step: 2903, Loss: 1.0332
Epoch: 008, Step: 2904, Loss: 1.0901
Epoch: 008, Step: 2905, Loss: 1.0825
Epoch: 008, Step: 2906, Loss: 1.0369
Epoch: 008, Step: 2907, Loss: 0.9497
Epoch: 008, Step: 2908, Loss: 0.9347
Epoch: 008, Step: 2909, Loss: 0.9424
Epoch: 008, Step: 2910, Loss: 0.8714
Epoch: 008, Step: 2911, Loss: 1.0795
Epoch: 008, Step: 2912, Loss: 1.0676
Epoch: 008, Step: 2913, Loss: 0.8373
Epoch: 008, Step: 2914, Loss: 0.9044
Epoch: 008, Step: 2915, Loss: 0.8956
Epoch: 008, Step: 2916, Loss: 0.9359
Epoch: 008, Step: 2917, 

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
def test_classification(model, test_loader):
    model.eval()
    predictions = []
    true_classes = []

    with torch.no_grad():
        for i, batch in enumerate(test_loader):
            x_dict = batch.x_dict
            edge_index_dict = batch.edge_index_dict
            test_mask = batch['products'].test_mask
            data_y = batch['products'].y

            out = model(x_dict, edge_index_dict)

            preds = out['products'][test_mask].argmax(dim=1)
            predictions.append(preds)
            true_classes.append(data_y[test_mask])

        predictions = torch.cat(predictions, dim=0)
        true_classes = torch.cat(true_classes, dim=0)

        print(f'Accuracy: {accuracy_score(true_classes.cpu(), predictions.cpu()):.2f}')
        print(f'Precision: {precision_score(true_classes.cpu(), predictions.cpu(), average="weighted"):.2f}')
        print(f'Recall: {recall_score(true_classes.cpu(), predictions.cpu(), average="weighted"):.2f}')
        print(f'F1-Score: {f1_score(true_classes.cpu(), predictions.cpu(), average="weighted"):.2f}')

In [ ]:
test_classification(model=model, test_loader=test_loader)

Accuracy: 0.63
Precision: 0.48


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Recall: 0.63
F1-Score: 0.52


# Задача 2

In [ ]:
import torch
from torch.optim import SGD
from torch.optim import Adam
from torch.utils.data import DataLoader
from torch_geometric.nn import LightGCN
from torch_geometric.datasets import IMDB
from torch_geometric.transforms import RandomLinkSplit

In [ ]:
from torch.nn.functional import mse_loss
from torch_geometric.nn import to_hetero
from torch_geometric.nn import Linear, SAGEConv


class GNNEncoder(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv((-1, -1), hidden_channels)
        self.conv2 = SAGEConv((-1, -1), out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x


class EdgeDecoder(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.lin1 = Linear(2 * hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, 1)

    def forward(self, z_dict, edge_label_index):
        row, col = edge_label_index
        z = torch.cat([z_dict['products'][row], z_dict['customers'][col]], dim=-1)

        z = self.lin1(z).relu()
        z = self.lin2(z)
        return z.view(-1)


class Model(torch.nn.Module):
    def __init__(self, hidden_channels, data):
        super().__init__()
        self.encoder = GNNEncoder(hidden_channels, hidden_channels)
        self.encoder = to_hetero(self.encoder, data.metadata(), aggr='sum')
        self.decoder = EdgeDecoder(hidden_channels)

    def forward(self, x_dict, edge_index_dict, edge_label_index):
        z_dict = self.encoder(x_dict, edge_index_dict)
        return self.decoder(z_dict, edge_label_index)



def train_link_prediction(model, train_data, val_data, optimizer, epochs=5):
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        pred = model(train_data.x_dict, train_data.edge_index_dict,
                     train_data['products', 'customers'].edge_label_index)
        target = train_data['products', 'customers'].edge_label
        loss = mse_loss(pred, target)
        loss.backward()
        optimizer.step()

        model.eval()
        pred = model(val_data.x_dict, val_data.edge_index_dict,
                     val_data['products', 'customers'].edge_label_index)
        pred = pred.clamp(min=0, max=5)
        target = val_data['products', 'customers'].edge_label.float()
        val_loss = mse_loss(pred, target).sqrt()

        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val Loss: {val_loss:.4f}')

In [ ]:
train_val_test_split = RandomLinkSplit(num_val=0.2,
                                       num_test=0.2,
                                       add_negative_train_samples=True,
                                       edge_types=[('products', 'similar_to', 'products'), ('products', 'reviewed_by', 'customers')],
                                       rev_edge_types=[('products', 'similar_to', 'products'), ('customers', 'reviewed_by', 'products')])

In [ ]:
train_data, val_data, test_data = train_val_test_split(data)

In [ ]:
train_data

HeteroData(
  products={
    x=[120120, 10],
    y=[120120],
    train_mask=[120120],
    test_mask=[120120],
    val_mask=[120120],
  },
  customers={ x=[257068, 10] },
  (products, similar_to, products)={
    edge_index=[2, 70740],
    edge_label=[141480],
    edge_label_index=[2, 141480],
  },
  (products, reviewed_by, customers)={
    edge_index=[2, 303465],
    edge_attr=[303465],
    edge_label=[606930],
    edge_label_index=[2, 606930],
  },
  (customers, reviewed_by, products)={}
)

In [ ]:
model_lp = Model(hidden_channels=196, data=data)

In [ ]:
optimizer_adam = Adam(model_lp.parameters(), lr=0.001)

In [ ]:
train_link_prediction(model=model_lp,
                      train_data=train_data,
                      val_data=val_data,
                      optimizer=optimizer_adam,
                      epochs=10)

Epoch: 000, Loss: 0.2212, Val Loss: 0.4710
Epoch: 001, Loss: 0.1807, Val Loss: 0.4279
Epoch: 002, Loss: 0.1281, Val Loss: 0.4409
Epoch: 003, Loss: 0.1342, Val Loss: 0.4400
Epoch: 004, Loss: 0.1366, Val Loss: 0.4043
Epoch: 005, Loss: 0.1125, Val Loss: 0.3746
Epoch: 006, Loss: 0.0994, Val Loss: 0.3711
Epoch: 007, Loss: 0.1087, Val Loss: 0.3739
Epoch: 008, Loss: 0.1167, Val Loss: 0.3682
Epoch: 009, Loss: 0.1094, Val Loss: 0.3671


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
def test_link_prediction(model, test_data):
    model.eval()  # Set model to evaluation mode

    with torch.no_grad():  # Disable gradient computation
        # Predict edge weights for the test set
        pred = model(test_data.x_dict, test_data.edge_index_dict,
                     test_data['products', 'customers'].edge_label_index)

        # Clamp predictions to the valid range (e.g., ratings between 0 and 5)
        pred = pred.clamp(min=0, max=5)

        # Convert predictions to discrete classes (round to nearest integer)
        pred_classes = pred.round().long()

        # True labels (ground truth)
        target = test_data['products', 'customers'].edge_label.long()

        # Compute metrics
        accuracy = accuracy_score(target.cpu(), pred_classes.cpu())
        precision = precision_score(target.cpu(), pred_classes.cpu(), average='weighted', zero_division=0)
        recall = recall_score(target.cpu(), pred_classes.cpu(), average='weighted', zero_division=0)
        f1 = f1_score(target.cpu(), pred_classes.cpu(), average='weighted', zero_division=0)

        # Print metrics
        print(f'Accuracy: {accuracy:.2f}')
        print(f'Precision: {precision:.2f}')
        print(f'Recall: {recall:.2f}')
        print(f'F1: {f1:.2f}')

In [ ]:
test_link_prediction(model_lp, test_data)

Accuracy: 0.85
Precision: 0.86
Recall: 0.85
F1: 0.85


# Задача 3

In [ ]:
import torch


def train(dataset, train_loader, model, optimizer, num_products, num_customers, epochs=1):
    for epoch in range(epochs):
        total_loss, total_examples = 0, 0

        for node_ids in train_loader:
            pos_edge_label_index = dataset.edge_index[:, node_ids]
            neg_edge_label_index = torch.stack([pos_edge_label_index[0],
                                                torch.randint(num_products, num_products + num_customers,
                                                              (node_ids.numel(),))],
                                               dim=0)
            edge_label_index = torch.cat([pos_edge_label_index, neg_edge_label_index], dim=1)

            optimizer.zero_grad()

            pos_rank, neg_rank = model(dataset.edge_index, edge_label_index).chunk(2)

            loss = model.recommendation_loss(pos_rank, neg_rank, node_id=edge_label_index.unique())
            loss.backward()
            optimizer.step()

            total_loss += float(loss) * pos_rank.numel()
            total_examples += pos_rank.numel()

            print(f'Epoch: {epoch:03d}, Loss: {total_loss / total_examples:.4f}')

In [ ]:
train_val_test_split = RandomLinkSplit(num_val=0.0,
                                       num_test=0.2,
                                       add_negative_train_samples=True,
                                       edge_types=[('products', 'similar_to', 'products'), ('products', 'reviewed_by', 'customers')],
                                       rev_edge_types=[('products', 'similar_to', 'products'), ('customers', 'reviewed_by', 'products')])

In [ ]:
train_data, val_data, test_data = train_val_test_split(data)

In [ ]:
train_data2 = train_data.to_homogeneous()

In [ ]:
model_lightgcn = LightGCN(num_nodes=train_data2.num_nodes,
                          embedding_dim=128,
                          num_layers=1)

In [ ]:
data_loader = DataLoader(range(train_data2.edge_index.size(1)),
                         shuffle=True,
                         batch_size=16)

In [ ]:
opt = Adam(model_lightgcn.parameters(), lr=0.001)

In [ ]:
train(dataset=train_data2,
      train_loader=data_loader,
      model=model_lightgcn,
      optimizer=opt,
      num_products=data['products'].num_nodes,
      num_customers=data['customers'].num_nodes,
      epochs=1)

Epoch: 000, Loss: 0.6913
Epoch: 000, Loss: 0.6917
Epoch: 000, Loss: 0.6919
Epoch: 000, Loss: 0.6918
Epoch: 000, Loss: 0.6918
Epoch: 000, Loss: 0.6916
Epoch: 000, Loss: 0.6917
Epoch: 000, Loss: 0.6912
Epoch: 000, Loss: 0.6911
Epoch: 000, Loss: 0.6911
Epoch: 000, Loss: 0.6911
Epoch: 000, Loss: 0.6911
Epoch: 000, Loss: 0.6911
Epoch: 000, Loss: 0.6909
Epoch: 000, Loss: 0.6909
Epoch: 000, Loss: 0.6903
Epoch: 000, Loss: 0.6897
Epoch: 000, Loss: 0.6898
Epoch: 000, Loss: 0.6897
Epoch: 000, Loss: 0.6897
Epoch: 000, Loss: 0.6894
Epoch: 000, Loss: 0.6889
Epoch: 000, Loss: 0.6890
Epoch: 000, Loss: 0.6891
Epoch: 000, Loss: 0.6889
Epoch: 000, Loss: 0.6888
Epoch: 000, Loss: 0.6889
Epoch: 000, Loss: 0.6890
Epoch: 000, Loss: 0.6888
Epoch: 000, Loss: 0.6883
Epoch: 000, Loss: 0.6880
Epoch: 000, Loss: 0.6877
Epoch: 000, Loss: 0.6874
Epoch: 000, Loss: 0.6871
Epoch: 000, Loss: 0.6869
Epoch: 000, Loss: 0.6870
Epoch: 000, Loss: 0.6868
Epoch: 000, Loss: 0.6867
Epoch: 000, Loss: 0.6867
Epoch: 000, Loss: 0.6860


In [ ]:
def get_model_embeddings(model):
    if hasattr(model, 'embedding'):
        embeddings = model.embedding.weight
        print(f"Embeddings found: {embeddings.shape}")
        return embeddings
    else:
        print("No direct embeddings found in model.")
        return None

embeddings_all_nodes = get_model_embeddings(model_lightgcn)


Embeddings found: torch.Size([17576, 128])


In [ ]:
dataset2 = data.to_homogeneous()

In [ ]:
import torch
import torch.nn.functional as F


def calculate_metrics_for_embedded_nodes(embeddings, edge_index, k_values=[1, 5, 10]):
    num_nodes = embeddings.shape[0]
    metrics = {k: {'recall': 0, 'precision': 0, 'f1': 0} for k in k_values}

    similarity_matrix = torch.matmul(embeddings, embeddings.t())

    positive_edges = edge_index.t().cpu().numpy()

    for k in k_values:
        for node in range(num_nodes):
            scores = similarity_matrix[node]

            sorted_nodes = torch.argsort(scores, descending=True).cpu().numpy()

            top_k_recommendations = sorted_nodes[sorted_nodes != node][:k]

            true_positives = sum([1 for n in top_k_recommendations if n in positive_edges[node]])
            recall = true_positives / k
            precision = true_positives / len(top_k_recommendations)
            f1 = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0

            metrics[k]['recall'] += recall
            metrics[k]['precision'] += precision
            metrics[k]['f1'] += f1

    for k in k_values:
        metrics[k]['recall'] /= num_nodes
        metrics[k]['precision'] /= num_nodes
        metrics[k]['f1'] /= num_nodes

    return metrics

metrics = calculate_metrics_for_embedded_nodes(embeddings_all_nodes, dataset2.edge_index, k_values=[1, 5, 10])

for k in [1, 5, 10]:
    print(f"Recall@{k}: {metrics[k]['recall']:.4f}, Precision@{k}: {metrics[k]['precision']:.4f}, F1@{k}: {metrics[k]['f1']:.4f}")



Recall@1: 0.1273, Precision@1: 0.1273, F1@1: 0.1273
Recall@5: 0.0440, Precision@5: 0.0440, F1@5: 0.0440
Recall@10: 0.0266, Precision@10: 0.0266, F1@10: 0.0266
